## Import Libraries, ignore warnings and set working directories

In [4]:
import pandas as pd
import yfinance as yf
import os

#dont show warnings
import warnings
warnings.filterwarnings("ignore")

data_directory = os.getcwd() + "/data/"
data_directory

ModuleNotFoundError: No module named 'yfinance'

In [3]:
import sys
environment_location = sys.prefix
environment_location

'/Users/phitzi/anaconda3'